In [10]:
%pip install -q youtube-transcript-api langchain-community langchain-google-genai faiss-cpu tiktoken python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install langchain

  Using cached langchain-1.0.2-py3-none-any.whl.metadata (4.7 kB)
  Using cached langgraph-1.0.1-py3-none-any.whl.metadata (7.4 kB)
  Using cached langgraph_checkpoint-3.0.0-py3-none-any.whl.metadata (4.2 kB)
  Using cached langgraph_prebuilt-1.0.1-py3-none-any.whl.metadata (5.0 kB)
  Using cached langgraph_sdk-0.2.9-py3-none-any.whl.metadata (1.5 kB)
  Using cached xxhash-3.6.0-cp312-cp312-win_amd64.whl.metadata (13 kB)
  Using cached ormsgpack-1.11.0-cp312-cp312-win_amd64.whl.metadata (1.2 kB)
Using cached langchain-1.0.2-py3-none-any.whl (107 kB)
Using cached langgraph-1.0.1-py3-none-any.whl (155 kB)
Using cached langgraph_checkpoint-3.0.0-py3-none-any.whl (46 kB)
Using cached langgraph_prebuilt-1.0.1-py3-none-any.whl (28 kB)
Using cached langgraph_sdk-0.2.9-py3-none-any.whl (56 kB)
Using cached xxhash-3.6.0-cp312-cp312-win_amd64.whl (31 kB)
Using cached ormsgpack-1.11.0-cp312-cp312-win_amd64.whl (112 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain_community.document_loaders import YoutubeLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
import os

In [7]:
video_url = "https://www.youtube.com/watch?v=Gfr50f6ZBvo"

try:
    # Initialize and load transcript
    loader = YoutubeLoader.from_youtube_url(video_url, add_video_info=False)
    docs = loader.load()

    # Combine all chunks into a single string
    transcript = " ".join(doc.page_content for doc in docs)

except Exception as e:
    print("Transcript could not be loaded:", e)

In [14]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1700, chunk_overlap=200)
chunks = splitter.create_documents([transcript])

In [15]:
len(chunks)

90

In [16]:
load_dotenv()
key = os.getenv("GOOGLE_API_KEY")

In [22]:
import time 
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001", google_api_key=key)

for i in range(0,len(chunks),10):
    if(i==0):
        vectorstore = FAISS.from_documents(chunks[i:i+10], embedding=embeddings)
    else:
        vectorstore.add_documents(chunks[i:i+10])
    print(f"Added documents {i} to {i+10}")
    time.sleep(10)

Added documents 0 to 10
Added documents 10 to 20
Added documents 20 to 30
Added documents 30 to 40
Added documents 40 to 50
Added documents 50 to 60
Added documents 60 to 70
Added documents 70 to 80
Added documents 80 to 90


In [23]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [24]:
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000020951BE25D0>, search_kwargs={'k': 4})

In [25]:
retriever.invoke('What is deepmind')

[Document(id='94789430-cdb5-4119-bda6-9b9372a8d23d', metadata={}, page_content="the the entertainment right you're actually actively involved as an as a as an agent so i think that's what makes it in some ways can be more visceral than other other mediums like you know films and books so the second so that was you know designing ai and games and then the third use uh uh i've we've used of ai is in deep mind from the beginning which is using games as a testing ground for proving out ai algorithms and developing ai algorithms and that was a that was a sort of um a core component of our vision at the start of deepmind was that we would use games very heavily uh as our main testing ground certainly to begin with um because it's super efficient to use games and also you know it's very easy to have metrics to see how well your systems are improving and what direction your ideas are going in and whether you're making incremental improvements and because those games are often rooted in somethi

In [ ]:
llm=ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", 
    google_api_key=key, 
    temperature=0.2
)

In [27]:
prompt=PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.
      
      {context}

      Question: {question}
    """,
    input_variables = ['context', 'question']
)


In [28]:
question          = "is the topic of nuclear fusion discussed in this video? if yes then what was discussed"
retrieved_docs    = retriever.invoke(question)

In [29]:
retrieved_docs

[Document(id='ab7d9834-db90-4c87-9fa9-533de996f8a4', metadata={}, page_content="problem is of course they can't react in the moment to something the plasma's doing that they have to be hard-coded and again knowing that that's normally our go-to solution is we would like to learn that instead and they also had a simulator of these plasma so there were lots of criteria that matched what we we like to to to use so can ai eventually solve nuclear fusion well so we with this problem and we published it in a nature paper last year uh we held the fusion that we held the plasma in specific shapes so actually it's almost like carving the plasma into different shapes and control and hold it there for the record amount of time so um so that's one of the problems of of fusion sort of um solved so i have a controller that's able to no matter the shape uh contain it continue yeah contain it and hold it in structure and there's different shapes that are better for for the energy productions called dr

In [30]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"problem is of course they can't react in the moment to something the plasma's doing that they have to be hard-coded and again knowing that that's normally our go-to solution is we would like to learn that instead and they also had a simulator of these plasma so there were lots of criteria that matched what we we like to to to use so can ai eventually solve nuclear fusion well so we with this problem and we published it in a nature paper last year uh we held the fusion that we held the plasma in specific shapes so actually it's almost like carving the plasma into different shapes and control and hold it there for the record amount of time so um so that's one of the problems of of fusion sort of um solved so i have a controller that's able to no matter the shape uh contain it continue yeah contain it and hold it in structure and there's different shapes that are better for for the energy productions called droplets and and and so on so um so that was huge and now we're looking we're tal

In [31]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [32]:
answer = llm.invoke(final_prompt)
print(answer.content)

Yes, the topic of nuclear fusion is discussed in the video.

What was discussed:
*   AI is being applied to help solve nuclear fusion challenges.
*   A specific problem in fusion, controlling and holding plasma in specific shapes (almost like carving it) for record amounts of time, was addressed using AI (deep reinforcement learning). This is considered one of the "bottleneck problems" of fusion.
*   Plasma control is crucial because plasma is extremely hot (millions of degrees Celsius, hotter than the sun) and unstable, requiring containment by powerful superconducting magnetic fields.
*   AI can predict plasma behavior and adjust the magnetic field within milliseconds, which traditional hard-coded controllers cannot do in the moment.
*   The work involved collaboration with EPFL in Switzerland, using their test reactor.
*   Fusion still faces many challenges, including physics, material science, and engineering.
*   The strategy is to identify bottleneck problems in fusion that are a

# Building the Chain

In [33]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [34]:
def format_docs(retrieved_docs):
    context_text= "\n\n".join(doc.page_content for doc in retrieved_docs)
    return context_text

In [36]:
parallel_chain=RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [37]:
parallel_chain.invoke("Who is Demis")

{'context': "thanks for listening to this conversation with demas establish to support this podcast please check out our sponsors in the description and now let me leave you with some words from edskar dykstra computer science is no more about computers than astronomy is about telescopes thank you for listening and hope to see you next time\n\nthe following is a conversation with demus hasabis ceo and co-founder of deepmind a company that has published and builds some of the most incredible artificial intelligence systems in the history of computing including alfred zero that learned all by itself to play the game of gold better than any human in the world and alpha fold two that solved protein folding both tasks considered nearly impossible for a very long time demus is widely considered to be one of the most brilliant and impactful humans in the history of artificial intelligence and science and engineering in general this was truly an honor and a pleasure for me to finally sit down 

In [38]:
parser=StrOutputParser()

In [39]:
main_chain=parallel_chain | prompt | llm | parser

In [40]:
main_chain.invoke("Summarize the video")

"The conversation covers several topics:\n\n1.  **AI and Human Interaction**: The speaker discusses the idea of AI systems getting frustrated and the belief that true intelligence should be able to explain complex concepts simply. They also touch upon human enhancement through compute devices like phones and potential future technologies like Neuralink.\n2.  **Existence of Alien Civilizations**: Based on current evidence and the lack of signals from programs like SETI, the speaker's personal opinion is that humanity is likely alone in the universe.\n3.  **AI Development Methodology (DeepMind's Alpha series)**: The discussion highlights the strategy of letting learning flow end-to-end in AI systems, starting with small learning components and expanding them. This is exemplified by the evolution of AlphaGo (trained for Go) to AlphaGo Zero (learned without human games), then AlphaZero (generalized to any two-player game without specific rules), and finally MuZero (which doesn't even need 